In [1]:
from pyspark.sql import SparkSession

spark= (SparkSession.builder.appName("Capstone")\
       .config("hive.metastore.uris","thrift://ip-10-1-2-24.ap-south-1.compute.internal:9083")
       .enableHiveSupport().getOrCreate())

In [2]:
spark

In [3]:
from pyspark.sql import functions as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from pyspark.sql.window import Window
# from pyspark.sql.functions import * #lot of functions available here
from pyspark.sql.types import IntegerType, StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.sql.types import *
from pyspark.sql.functions import when, count

1. A list showing employee number, last name, first name, sex, and salary for each employee

In [4]:
spark.sql("""select emp123.emp_no, emp123.last_name, emp123.first_name, 
emp123.sex,
         Salary.salary from emp123 join Salary on emp123.emp_no == Salary.emp_no""").show(5,truncate=False)

+------+---------+----------+---+------+
|emp_no|last_name|first_name|sex|salary|
+------+---------+----------+---+------+
|473302|Zallocco |Hideyuki  |M  |40000 |
|475053|Delgrande|Byong     |F  |53422 |
|57444 |Babb     |Berry     |F  |48973 |
|421786|Verhoeff |Xiong     |M  |40000 |
|282238|Baumann  |Abdelkader|F  |40000 |
+------+---------+----------+---+------+
only showing top 5 rows



2. A list showing first name, last name, and hire date for employees who were hired in 1986.

In [5]:
spark.sql("""select emp123.first_name, emp123.last_name,emp123.hire_date from emp123 
          where hire_date BETWEEN '1/1/1986' AND '12/31/1986' """).show(20,truncate=False)

+----------+-----------+----------+
|first_name|last_name  |hire_date |
+----------+-----------+----------+
|Xiong     |Verhoeff   |11/26/1987|
|Abdelkader|Baumann    |1/18/1991 |
|Eran      |Cusworth   |11/14/1986|
|Xudong    |Samarati   |11/13/1985|
|Lihong    |Magliocco  |10/23/1993|
|Shuichi   |Tyugu      |1/17/1995 |
|Bojan     |Zallocco   |10/14/1986|
|Bilhanan  |Wuwongse   |10/6/1993 |
|Elliott   |Perl       |10/29/1987|
|Nechama   |Copas      |10/27/1987|
|Pantung   |Cools      |1/28/1994 |
|Hairong   |Schaar     |12/24/1987|
|Mohit     |Speek      |1/14/1986 |
|Bader     |Chinal     |1/8/1990  |
|Gad       |Nollmann   |10/1/1985 |
|Munehiko  |Janocha    |12/11/1988|
|Shaowen   |Krone      |11/18/1990|
|Rosalie   |Rousseau   |11/7/1998 |
|Hitomi    |Gunderson  |11/22/1987|
|Leni      |Pusterhofer|1/29/1995 |
+----------+-----------+----------+
only showing top 20 rows



3. A list showing the manager of each department with the following information: department number, department name, the manager's employee number, last name, first name.

In [6]:
spark.sql("""select depart.dept_no, depart.dept_name, emp123.emp_no,
         emp123.last_name,emp123.first_name from depart join
         depart_manager on depart.dept_no == depart_manager.dept_no
         join emp123 on depart_manager.emp_no == emp123.emp_no""").show(10,truncate=False)

+-------+--------------------+------+------------+-----------+
|dept_no|dept_name           |emp_no|last_name   |first_name |
+-------+--------------------+------+------------+-----------+
|d009   |"Customer Service"  |111877|Spinelli    |Xiaobin    |
|d008   |"Research"          |111534|Kambil      |Hilary     |
|d006   |"Quality Management"|110765|Hofmeyr     |Rutger     |
|d004   |"Production"        |110420|Ghazalie    |Oscar      |
|d006   |"Quality Management"|110725|Onuegbe     |Peternela  |
|d001   |"Marketing"         |110022|Markovitch  |Margareta  |
|d007   |"Sales"             |111035|Kaelbling   |Przemyslawa|
|d005   |"development"       |110511|Hagimont    |DeForest   |
|d005   |"development"       |110567|DasSarma    |Leon       |
|d003   |"Human Resources"   |110183|Ossenbruggen|Shirish    |
+-------+--------------------+------+------------+-----------+
only showing top 10 rows



4. A list showing the department of each employee with the following information: employee number, last name, first name, and department name.

In [7]:
spark.sql("""select depart.dept_name, emp123.emp_no,
         emp123.last_name,emp123.first_name from depart join
         depart_manager on depart.dept_no == depart_manager.dept_no
         join emp123 on depart_manager.emp_no == emp123.emp_no""").show(10,truncate=False)

+--------------------+------+------------+-----------+
|dept_name           |emp_no|last_name   |first_name |
+--------------------+------+------------+-----------+
|"Customer Service"  |111877|Spinelli    |Xiaobin    |
|"Research"          |111534|Kambil      |Hilary     |
|"Quality Management"|110765|Hofmeyr     |Rutger     |
|"Production"        |110420|Ghazalie    |Oscar      |
|"Quality Management"|110725|Onuegbe     |Peternela  |
|"Marketing"         |110022|Markovitch  |Margareta  |
|"Sales"             |111035|Kaelbling   |Przemyslawa|
|"development"       |110511|Hagimont    |DeForest   |
|"development"       |110567|DasSarma    |Leon       |
|"Human Resources"   |110183|Ossenbruggen|Shirish    |
+--------------------+------+------------+-----------+
only showing top 10 rows



5 A list showing first name, last name, and sex for employees whose first name is "Hercules" and last names begin with "B.“

In [8]:
spark.sql("""select emp123.first_name,emp123.last_name,emp123.sex
         from emp123 where first_name = 'Hercules' and last_name like 'B%'""").show(10,truncate=False)

+----------+-------------+---+
|first_name|last_name    |sex|
+----------+-------------+---+
|Hercules  |Baer         |M  |
|Hercules  |Biron        |F  |
|Hercules  |Birge        |F  |
|Hercules  |Berstel      |F  |
|Hercules  |Bernatsky    |M  |
|Hercules  |Bail         |F  |
|Hercules  |Bodoff       |M  |
|Hercules  |Benantar     |F  |
|Hercules  |Basagni      |M  |
|Hercules  |Bernardinello|F  |
+----------+-------------+---+
only showing top 10 rows



6. A list showing all employees in the Sales department, including their employee number, last name, first name, and department name.

In [9]:
spark.sql("""select emp123.last_name,emp123.first_name,depart.dept_name from dept_employee
    join emp123 on dept_employee.emp_no == emp123.emp_no join depart
    on dept_employee.dept_no == depart.dept_no where depart.dept_name = 'Sales'""").show(5, truncate=False)

+---------+----------+---------+
|last_name|first_name|dept_name|
+---------+----------+---------+
+---------+----------+---------+



7.A list showing all employees in the Sales and Development departments, including their employee number, last name, first name, and department name.

In [10]:
spark.sql("""select emp123.last_name,emp123.first_name,depart.dept_name from dept_employee
    join emp123 on dept_employee.emp_no == emp123.emp_no join depart
    on dept_employee.dept_no == depart.dept_no where depart.dept_name = 'Sales'
    or depart.dept_name= 'Development'""").show(5, truncate=False)

+---------+----------+---------+
|last_name|first_name|dept_name|
+---------+----------+---------+
+---------+----------+---------+



8.A list showing the frequency count of employee last names, in descending order. ( i.e., how many employees share each last name

In [11]:
x= spark.sql("select* from emp123")

In [43]:
x.groupBy('last_name').count().show()

+----------------+-----+
|       last_name|count|
+----------------+-----+
|       Delgrossi|  188|
|         Rullman|  171|
|        Hagimont|  164|
|         Lanteri|  173|
|          Nanard|  194|
|          Saoudi|  176|
|   Beutelspacher|  180|
|          Butner|  193|
|          Stille|  165|
|          Aamodt|  205|
|Sankaranarayanan|  183|
|         Kleiser|  193|
|       Besselaar|  188|
|         Solovay|  178|
|          Ritcey|  156|
|      Hinsberger|  187|
|            Lund|  170|
|     Shokrollahi|  179|
|      Taubenfeld|  200|
|            Gips|  167|
+----------------+-----+
only showing top 20 rows



9.Histogram to show the salary distribution among the employees

In [13]:
y = spark.sql("select * from emp123 join Salary on emp123.emp_no == Salary.emp_no")

10. Bar graph to show the Average salary per title (designation)

12: 1

In [14]:
a= spark.sql("select * from emp123 join Salary on emp123.emp_no ==Salary.emp_no")

In [15]:
a.groupBy('sex').sum('salary').show()

+---+-----------+
|sex|sum(salary)|
+---+-----------+
|  F| 6357161017|
|  M| 9535330016|
+---+-----------+



12: 2

In [16]:
a.groupBy('left_').sum('salary').show()

+-----+-----------+
|left_|sum(salary)|
+-----+-----------+
|    0|14316882768|
|    1| 1575608265|
+-----+-----------+



12:3

In [17]:
b= spark.sql("select * from depart_manager join emp123 on depart_manager.emp_no == emp123.emp_no")

In [18]:
b.groupBy("dept_no").count().show()

+-------+-----+
|dept_no|count|
+-------+-----+
|   d005|    2|
|   d009|    4|
|   d003|    2|
|   d001|    2|
|   d007|    2|
|   d004|    4|
|   d002|    2|
|   d006|    4|
|   d008|    2|
+-------+-----+



12:4

In [19]:
c= spark.sql("select * from depart join dept_employee on depart.dept_no==dept_employee.dept_no")

In [20]:
c.groupBy('dept_name').count().show()

+--------------------+-----+
|           dept_name|count|
+--------------------+-----+
|"Quality Management"|20117|
|        "Production"|73485|
|       "development"|85707|
|          "Research"|21126|
|             "Sales"|52245|
|           "Finance"|17346|
|   "Human Resources"|17786|
|         "Marketing"|20211|
|  "Customer Service"|23580|
+--------------------+-----+



# A.Read the data from Hive tables

In [21]:
# Reading the data from hive table 

In [22]:

spark.sql("select * from depart").show()

+-------+--------------------+
|dept_no|           dept_name|
+-------+--------------------+
|   d001|         "Marketing"|
|   d002|           "Finance"|
|   d003|   "Human Resources"|
|   d004|        "Production"|
|   d005|       "development"|
|   d006|"Quality Management"|
|   d007|             "Sales"|
|   d008|          "Research"|
|   d009|  "Customer Service"|
+-------+--------------------+



In [23]:
spark.sql("select * from Salary").show()

+------+------+
|emp_no|salary|
+------+------+
| 10001| 60117|
| 10002| 65828|
| 10003| 40006|
| 10004| 40054|
| 10005| 78228|
| 10006| 40000|
| 10007| 56724|
| 10008| 46671|
| 10009| 60929|
| 10010| 72488|
| 10011| 42365|
| 10012| 40000|
| 10013| 40000|
| 10014| 46168|
| 10015| 40000|
| 10016| 70889|
| 10017| 71380|
| 10018| 55881|
| 10019| 44276|
| 10020| 40000|
+------+------+
only showing top 20 rows



In [24]:
spark.sql("select * from emp123").show()

+------+------------+----------+----------+----------+---+----------+--------------+-----------------------+-----+----------+
|emp_no|emp_title_id|birth_date|first_name| last_name|sex| hire_date|no_of_projects|last_performance_rating|left_| last_date|
+------+------------+----------+----------+----------+---+----------+--------------+-----------------------+-----+----------+
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
+------+------------+----------+----------+----------+---+----------+--------------+-----------------------+-----+----------+
only showing top 20 rows



In [25]:
spark.sql("select * from depart_manager").show()

+-------+------+
|dept_no|emp_no|
+-------+------+
|   d001|110022|
|   d001|110039|
|   d002|110085|
|   d002|110114|
|   d003|110183|
|   d003|110228|
|   d004|110303|
|   d004|110344|
|   d004|110386|
|   d004|110420|
|   d005|110511|
|   d005|110567|
|   d006|110725|
|   d006|110765|
|   d006|110800|
|   d006|110854|
|   d007|111035|
|   d007|111133|
|   d008|111400|
|   d008|111534|
+-------+------+
only showing top 20 rows



In [26]:
spark.sql("select * from dept_employee").show()

+------+-------+
|emp_no|dept_no|
+------+-------+
| 10001|   d005|
| 10002|   d007|
| 10003|   d004|
| 10004|   d004|
| 10005|   d003|
| 10006|   d005|
| 10007|   d008|
| 10008|   d005|
| 10009|   d006|
| 10010|   d004|
| 10010|   d006|
| 10011|   d009|
| 10012|   d005|
| 10013|   d003|
| 10014|   d005|
| 10015|   d008|
| 10016|   d007|
| 10017|   d001|
| 10018|   d004|
| 10018|   d005|
+------+-------+
only showing top 20 rows



In [27]:
spark.sql("select * from title").show()

+--------+------------------+
|title_id|             title|
+--------+------------------+
|   s0001|             Staff|
|   s0002|      Senior Staff|
|   e0001|Assistant Engineer|
|   e0002|          Engineer|
|   e0003|   Senior Engineer|
|   e0004|  Technique Leader|
|   m0001|           Manager|
+--------+------------------+



# B.You required to join all the tables at employee level

In [28]:
# Join all the tables 

In [29]:
df= spark.sql("""select emp123.emp_no, emp123.emp_title_id, emp123.birth_date, emp123.first_name,
    emp123.last_name, emp123.sex, emp123.hire_date, emp123.no_of_projects, emp123.last_performance_rating,
    emp123.left_,emp123.last_date, Salary.salary, title.title, dept_employee.dept_no,
    depart.dept_name from emp123 join dept_employee on emp123.emp_no == dept_employee.emp_no join title
    on emp123.emp_title_id == title.title_id join Salary on emp123.emp_no == Salary.emp_no join depart on 
    dept_employee.dept_no == depart.dept_no""")

In [30]:
df.printSchema()

root
 |-- emp_no: string (nullable = true)
 |-- emp_title_id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- no_of_projects: integer (nullable = true)
 |-- last_performance_rating: string (nullable = true)
 |-- left_: string (nullable = true)
 |-- last_date: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- title: string (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)



In [31]:
df.show(5)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+---------+------+---------------+-------+--------------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|last_performance_rating|left_|last_date|salary|          title|dept_no|           dept_name|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+---------+------+---------------+-------+--------------------+
| 40000|          Staff|   d002|           "Finance"|
| 53422|       Engineer|   d004|        "Production"|
| 48973|       Engineer|   d004|        "Production"|
| 40000|          Staff|   d003|   "Human Resources"|
| 40000|Senior Engineer|   d006|"Quality Management"|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+---------+------+---------------+-------+--------------------+
only showing top 5 rows



In [32]:
df.columns

['emp_no',
 'emp_title_id',
 'birth_date',
 'first_name',
 'last_name',
 'sex',
 'hire_date',
 'no_of_projects',
 'last_performance_rating',
 'left_',
 'last_date',
 'salary',
 'title',
 'dept_no',
 'dept_name']

In [33]:
#Creating the summary report of Countinous Variable
df.select(['no_of_projects','salary']).summary().show()

+-------+------------------+------------------+
|summary|    no_of_projects|            salary|
+-------+------------------+------------------+
|  count|            331603|            331603|
|   mean|5.5048567111877755| 52972.22543221865|
| stddev|2.8728285038833126|14299.003000805926|
|    min|                 1|             40000|
|    25%|                 3|             40000|
|    50%|                 6|             48688|
|    75%|                 8|             61761|
|    max|                10|            129492|
+-------+------------------+------------------+



# Data Preparation Encoding

In [34]:
df.show(10)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+----------+------+---------------+-------+--------------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|last_performance_rating|left_| last_date|salary|          title|dept_no|           dept_name|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+-----+----------+------+---------------+-------+--------------------+
| 40000|          Staff|   d002|           "Finance"|
| 53422|       Engineer|   d004|        "Production"|
| 48973|       Engineer|   d004|        "Production"|
| 40000|          Staff|   d003|   "Human Resources"|
| 40000|Senior Engineer|   d006|"Quality Management"|
| 40000|Senior Engineer|   d006|"Quality Management"|
| 56087|          Staff|   d003|   "Human Resources"|
| 40000|   Senior Staff|   d002|           "Finance"|
| 54816|       Engineer|   d006|"Qual

In [35]:
df.printSchema()

root
 |-- emp_no: string (nullable = true)
 |-- emp_title_id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- no_of_projects: integer (nullable = true)
 |-- last_performance_rating: string (nullable = true)
 |-- left_: string (nullable = true)
 |-- last_date: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- title: string (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)



In [36]:
#Encoding all categorical features
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer,OneHotEncoderEstimator,StandardScaler

In [37]:
df_new=df

In [38]:
continious_features =['emp_no','emp_title_id','birth_date','first_name','last_name','hire_date',
                      'no_of_projects','last_date','title','dept_no']
categorical_features= ['sex','last_performance_rating','salary','dept_name']

StringIndexer

In [39]:
SI_sex = StringIndexer(inputCol='sex', outputCol='sex_index')
SI_last_performance_rating= StringIndexer(inputCol='last_performance_rating', outputCol='last_performance_rating_index')
SI_salary=StringIndexer(inputCol='salary', outputCol='salary_index')
SI_dept_name=StringIndexer(inputCol='dept_name', outputCol='dept_name_index')

df_new=SI_sex.fit(df_new).transform(df_new)
df_new=SI_last_performance_rating.fit(df_new).transform(df_new)
df_new=SI_salary.fit(df_new).transform(df_new)
df_new=SI_dept_name.fit(df_new).transform(df_new)

df_new.select('sex','sex_index','last_performance_rating','last_performance_rating_index','salary','salary_index','dept_name','dept_name_index').show(10)


+---+---------+-----------------------+-----------------------------+------+------------+--------------------+---------------+
|sex|sex_index|last_performance_rating|last_performance_rating_index|salary|salary_index|           dept_name|dept_name_index|
+---+---------+-----------------------+-----------------------------+------+------------+--------------------+---------------+
|  M|      0.0|                      A|                          1.0| 40000|         0.0|           "Finance"|            8.0|
|  F|      1.0|                      C|                          2.0| 53422|      4384.0|        "Production"|            1.0|
|  F|      1.0|                      A|                          1.0| 48973|     20860.0|        "Production"|            1.0|
|  M|      0.0|                      C|                          2.0| 40000|         0.0|   "Human Resources"|            7.0|
|  F|      1.0|                      B|                          0.0| 40000|         0.0|"Quality Management"| 

One-Hot Encoder

In [40]:
#create object and specify input and output column
encoder= OneHotEncoderEstimator().setInputCols(['sex_index','last_performance_rating_index','salary_index','dept_name_index']).setOutputCols(['sex_vec','last_performance_rating_vec','salary_vec','dept_name_vec'])

In [41]:
df_new.select('sex_vec','last_performance_rating_vec','salary_vec','dept_name_vec').show(5)

AnalysisException: "cannot resolve '`sex_vec`' given input columns: [sex_index, default.emp123.sex, default.emp123.left_, last_performance_rating_index, dept_name_index, default.emp123.birth_date, default.salary.salary, salary_index, default.depart.dept_name, default.dept_employee.dept_no, default.emp123.last_name, default.emp123.no_of_projects, default.emp123.emp_no, default.emp123.first_name, default.title.title, default.emp123.emp_title_id, default.emp123.hire_date, default.emp123.last_performance_rating, default.emp123.last_date];;\n'Project ['sex_vec, 'last_performance_rating_vec, 'salary_vec, 'dept_name_vec]\n+- Project [emp_no#544, emp_title_id#545, birth_date#546, first_name#547, last_name#548, sex#549, hire_date#550, no_of_projects#551, last_performance_rating#552, left_#553, last_date#554, salary#560L, title#558, dept_no#556, dept_name#562, sex_index#858, last_performance_rating_index#894, salary_index#932, UDF(cast(dept_name#562 as string)) AS dept_name_index#972]\n   +- Pro

In [ ]:
#create vector from categorical columns
featureCols =['sex_vec','last_performance_rating_vec','salary_vec','dept_name_vec']

Vector Assembler

In [ ]:
Vector_assembler = VectorAssembler(inputCols = featureCols,outputCol = "features")

In [ ]:
caps_df= Vector_assembler.transform(df_new)

In [ ]:
caps_df.columns

Label Indexer

In [ ]:
caps_df= caps_df.withColumn('label', when(caps_df.left_ == '1',1).otherwise(0))

In [ ]:
caps_df.select('left_','label').show(5)

In [ ]:
caps_df.show(10)

TRain & Test

In [ ]:
train_df,test_df = caps_df.randomSplit([0.7,0.3], seed= 50)

In [ ]:
#Build the Linear Regression Model
from pyspark.ml.classification import LogisticRegression 

In [ ]:
logrg = LogisticRegression(featuresCol= 'features',labelCol='label',maxIter=5)

In [ ]:
lr = logrg.fit(train_df)

In [ ]:
lr.intercept

In [ ]:
lr.coefficients

In [ ]:
y_pred_test = lr.transform(test_df)

In [ ]:
y_pred_train = lr.transform(train_df)

In [ ]:
y_pred_train.select(['label',
 'rawPrediction',
 'probability',
 'prediction']).toPandas().head(10)

In [ ]:
evaluator = BinaryClassificationEvaluator()

In [ ]:
evaluator.evaluate(y_pred_test)

Accuracy

In [ ]:
y_pred_test.filter(y_pred_test.label == y_pred_test.prediction).count() / float(y_pred_test.count())

In [ ]:
    print(lr.explainParams())